## Repressors / Activators analysis

In [8]:
pip install bioframe

  Using cached bioframe-0.4.1-py2.py3-none-any.whl (114 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os,sys
import multiDGD
import numpy as np
import pandas as pd
#import mudata as md
import anndata as ad

In [2]:
# load some test data
model_dir = '/home/jovyan/mount/gdrive/DGD_multiomics/data/bonemarrow_model/'
os.listdir(model_dir)

['GSE194122_openproblems_neurips2021_multiome_BMMC_processed_withDataSplit.gene_names.csv',
 'GSE194122_openproblems_neurips2021_multiome_BMMC_processed_withDataSplit.h5ad',
 'GSE194122_openproblems_neurips2021_multiome_BMMC_processed_withDataSplit.h5ad.zip',
 'GSE194122_openproblems_neurips2021_multiome_BMMC_processed_withDataSplit.peaks.csv',
 'bonemarrow_atac_prediction',
 'dgd_bonemarrow_default_trained_and_tested.pt',
 'dgd_bonemarrow_default_trained_and_tested_hyperparameters.json',
 'notebooks']

In [3]:
data = ad.read_h5ad(model_dir + 'GSE194122_openproblems_neurips2021_multiome_BMMC_processed_withDataSplit.h5ad', backed=True)

## Get genomic ranges of peaks

In [6]:
def _bioframe_from_strings(pos_list: pd.Series):
    """
    Function to create bioframe compatible table from string of genomic position
    """
    # Chromosome and positions
    chr = pos_list.str.split(':').str.get(0)
    start = pd.Series(pos_list.str.split(':').str.get(1)
                      ).str.split('-').str.get(0)
    end = pd.Series(pos_list.str.split(':').str.get(1)
                    ).str.split('-').str.get(1)

    # Create ranges
    bioframe_df = pd.concat([chr, start, end], axis=1)
    bioframe_df.columns = ['chrom', 'start', 'end']
    bioframe_df.loc[:,['start', 'end']] = bioframe_df.loc[:,['start', 'end']].astype(int)
    return bioframe_df.convert_dtypes()

# Get ranges of all peaks
pos_list = data.var[data.var['feature_types'] == 'ATAC'].index.tolist()
pos_list = pd.Series(pos_list).str.replace("-", ":", 1)
peaks_df = _bioframe_from_strings(pos_list)
peaks_df['name'] = '.'
peaks_df.columns = peaks_df.columns.astype('str')

In [7]:
peaks_df.to_csv(model_dir + 'GSE194122_openproblems_neurips2021_multiome_BMMC_processed_withDataSplit.peaks.csv')

In [ ]:
## Save gene names
gene_names = data.var[data.var['feature_types'] == 'GEX']
gene_names.to_csv(model_dir + 'GSE194122_openproblems_neurips2021_multiome_BMMC_processed_withDataSplit.gene_names.csv')

In [3]:
peaks_df = pd.read_csv(model_dir + 'GSE194122_openproblems_neurips2021_multiome_BMMC_processed_withDataSplit.peaks.csv', index_col=0)
gene_names = pd.read_csv(model_dir + 'GSE194122_openproblems_neurips2021_multiome_BMMC_processed_withDataSplit.gene_names.csv', index_col=0) 

In [5]:
gene_names.index

Index(['AL627309.5', 'LINC01409', 'LINC01128', 'NOC2L', 'KLHL17', 'ISG15',
       'C1orf159', 'SDF4', 'B3GALT6', 'UBE2J2',
       ...
       'MT-ATP6', 'MT-CO3', 'MT-ND3', 'MT-ND4L', 'MT-ND4', 'MT-ND5', 'MT-ND6',
       'MT-CYB', 'AL592183.1', 'AC240274.1'],
      dtype='object', length=13431)

## Get genomic ranges of TF motifs

In [6]:
import pandas as pd
import bioframe
from pyjaspar import jaspardb

In [12]:
## Read list of repressors/activators
tf_df = pd.read_table('/home/jovyan/mount/gdrive/DGD_multiomics/data/TF_mode_domcke_2020.txt')
tf_df = tf_df[tf_df.Transcription_factor.isin(gene_names.index)].copy()
tf_df

,Transcription_factor,GO_term_mode_of_action
0,GFI1B,Repressor
1,BACH2,Repressor
2,PBX2,Activator
4,ARID5A,Repressor
6,HESX1,Repressor
...,...,...
222,PLAG1,Activator
230,RFX3,Activator
235,CEBPD,Activator
240,SRF,Activator


In [13]:
JASPAR_URL = 'http://expdata.cmmt.ubc.ca/JASPAR/downloads/UCSC_tracks/2022/hg38/'
CHROMS = [str(x) for x in range(1,23)] + ['X', 'Y']
CHROMS = ['chr'+x for x in CHROMS]

jdb_obj = jaspardb(release='JASPAR2022')

def get_TF_matches(args) -> pd.DataFrame:
    '''
    Get TF motif matches for each peak using bioframe
    '''
    peaks_df, tf_name = args
    # Get IDs for TF motifs
    tf_ids = [m.matrix_id for m in jdb_obj.fetch_motifs_by_name(tf_name)]
    if len([t for t in tf_ids if t.startswith("MA")]) == 0:
        raise ValueError(f'No motifs found for TF {tf_name}')
        
    # Load TF motifs ranges
    motifs_df = pd.DataFrame()
    for tf_id in tf_ids:
        if tf_id.startswith('MA'):
            file_url = f'{JASPAR_URL}/{tf_id}.tsv.gz'
            jaspar_df = bioframe.read_table(file_url, schema='jaspar',skiprows=1)
            motifs_df = pd.concat([motifs_df, jaspar_df])

    # Clean up
    motifs_df = motifs_df[motifs_df.chrom.isin(CHROMS)]
    motifs_df = motifs_df[['chrom', 'start', 'end', 'name']].drop_duplicates()
    
    ## Find overlaps
    peaks_motif_counts = bioframe.count_overlaps(peaks_df, motifs_df)
    peaks_motif_counts[f'{tf_name}_motifs'] = peaks_motif_counts['count'].copy()
    peaks_motif_counts[f'{tf_name}_motifs'] = np.where(peaks_motif_counts[tf_name + '_motifs'] > 0, 1, 0)

    return peaks_motif_counts[['chrom', 'start', 'end', 'name', f'{tf_name}_motifs' ]]

In [ ]:
import gc

motifs_in_peaks = pd.DataFrame()
n_tfs = tf_df['Transcription_factor'].shape[0]
for i,tf in enumerate(tf_df['Transcription_factor']):
    print(f'TF {i+1}/{n_tfs} - {tf}')
    try:
        peaks_motif_counts = get_TF_matches((peaks_df, tf))[[f'{tf}_motifs']]
    except ValueError:
        continue
    motifs_in_peaks = pd.concat([motifs_in_peaks,peaks_motif_counts ], axis=1)
    gc.collect()
    
motifs_in_peaks.index = peaks_df[['chrom', 'start', 'end']].astype(str).apply(lambda x: "-".join(x), axis=1)
motifs_in_peaks.write_csv(model_dir + 'GSE194122_openproblems_neurips2021_multiome_BMMC_processed_withDataSplit.motifs_in_peaks.csv')

TF 1/112 - GFI1B
TF 2/112 - BACH2
TF 3/112 - PBX2
TF 4/112 - ARID5A
TF 5/112 - HESX1
TF 6/112 - CUX1
TF 7/112 - ARID3A
TF 8/112 - BCL6
TF 9/112 - VENTX
TF 10/112 - ARID3B
TF 11/112 - NRL
TF 12/112 - TFAP4
TF 13/112 - STAT4
TF 14/112 - NFE2
TF 15/112 - FOXJ2
TF 16/112 - HSF2
TF 17/112 - MECOM
TF 18/112 - PBX3
TF 19/112 - HLTF
TF 20/112 - PKNOX1
TF 21/112 - MEF2D
TF 22/112 - NFAT5
TF 23/112 - ARNT
TF 24/112 - TFCP2
TF 25/112 - NFATC1
TF 26/112 - SPIB
TF 27/112 - TCFL5
TF 28/112 - MLXIP
TF 29/112 - MXI1
TF 30/112 - RORA
TF 31/112 - TFEB
TF 32/112 - KLF1
TF 33/112 - CREB5
TF 34/112 - TGIF1
TF 35/112 - NR2C2


---

In [ ]:
# load model from the saved checkpoint
model = multiDGD.DGD.load(data=data, save_dir=model_dir, model_name='dgd_bonemarrow_default_trained_and_tested')

### Gene2Peak

This feature performs in silico perturbations on the specified gene and predicts the changes in prediction on all output features.

Currently, we only support this being performed on the test data. See the tutorial on training and testing an anndata object for details on the model and test data.

In [3]:
# specify the gene we want to look at
gene_name = "ID2"

# and the samples we want to look at
test_set = data[data.obs["train_val_test"] == "test",:].copy()

In [4]:
predicted_changes, samples_of_interest = model.gene2peak(gene_name=gene_name, testset=test_set)

using 185 samples


In [6]:
delta_gex = predicted_changes[0]
delta_atac = predicted_changes[1]